# Timesfm Weekly

In [1]:
!pip install timesfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of googleapis-common-protos to determine which version is compatible with other requirements. This could take a while.
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.2/440.2 kB 28.5 MB/s eta

In [1]:
import pandas as pd
import timesfm

2024-09-12 06:23:37.365107: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64


In [2]:
data = pd.read_excel("weekly_data.xlsx")

In [3]:
data.head()

,date,warehouse,orders,holiday_name,holiday,shutdown,mini_shutdown,shops_closed,winter_school_holidays,school_holidays,blackout,mov_change,frankfurt_shutdown,precipitation,snow,user_activity_1,user_activity_2,id
0,2020-12-06,Prague_1,13479,0,0,0,0,0,0,0,0,0,0,0.0,0.0,3410,65082,Prague_1_2020-12-05Prague_1_2020-12-06
1,2020-12-13,Prague_1,49517,0,0,0,0,0,0,0,0,0,0,1.3,0.0,11976,227222,Prague_1_2020-12-07Prague_1_2020-12-08Prague_1...
2,2020-12-20,Prague_1,53288,0,0,0,0,0,0,0,0,0,0,0.0,0.0,12429,227107,Prague_1_2020-12-14Prague_1_2020-12-15Prague_1...
3,2020-12-27,Prague_1,45261,Christmas Eve2nd Christmas Day,2,0,0,1,0,0,0,0,0,7.0,0.0,10738,193645,Prague_1_2020-12-21Prague_1_2020-12-22Prague_1...
4,2021-01-03,Prague_1,45811,New Years Day,1,0,0,1,0,0,0,0,0,1.0,0.0,11916,220659,Prague_1_2020-12-28Prague_1_2020-12-29Prague_1...


In [4]:
data = data.drop(["holiday_name", 'shutdown', 'mini_shutdown', 'blackout', 'frankfurt_shutdown', "id", 'blackout', 'mov_change', 'frankfurt_shutdown','precipitation', 'snow', 'user_activity_1', 'user_activity_2'], axis=1)

In [5]:
#rename as per requirement of the forecast libraries
data = data.rename(columns={"warehouse": "unique_id", "date": "ds", "orders": "y"})

In [6]:
data.head()

,ds,unique_id,y,holiday,shops_closed,winter_school_holidays,school_holidays
0,2020-12-06,Prague_1,13479,0,0,0,0
1,2020-12-13,Prague_1,49517,0,0,0,0
2,2020-12-20,Prague_1,53288,0,0,0,0
3,2020-12-27,Prague_1,45261,2,1,0,0
4,2021-01-03,Prague_1,45811,1,1,0,0


In [7]:
data['ds'] = pd.to_datetime(data['ds'])

In [8]:

train = data[data['ds'] <= '2023-05-14']
test = data[data['ds'] > '2023-05-14']

In [9]:
Y_test_df = test.copy()
df = train.copy()

In [11]:
import timesfm
tfm = timesfm.TimesFm(
    context_len=512,
    horizon_len=44,
    input_patch_len=32,
    output_patch_len=128,
    num_layers=20,
    model_dims=1280,
    backend="cpu",
)
tfm.load_from_checkpoint(repo_id="google/timesfm-1.0-200m")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.73k [00:00<?, ?B/s]

(…)oint_1100000/descriptor/descriptor.pbtxt:   0%|          | 0.00/499 [00:00<?, ?B/s]

...

checkpoint:   0%|          | 0.00/814M [00:00<?, ?B/s]

Constructing model weights.


Constructed model weights in 4.72 seconds.
Restoring checkpoint from /root/.cache/huggingface/hub/models--google--timesfm-1.0-200m/snapshots/8775f7531211ac864b739fe776b0b255c277e2be/checkpoints.


ERROR:absl:For checkpoint version > 1.0, we require users to provide
          `train_state_unpadded_shape_dtype_struct` during checkpoint
          saving/restoring, to avoid potential silent bugs when loading
          checkpoints to incompatible unpadded shapes of TrainState.


Restored checkpoint in 2.06 seconds.
Jitting decoding.
Jitted decoding in 39.30 seconds.


In [12]:
timesfm_df = tfm.forecast_on_df(
    inputs=df,
    freq='W',
    value_name='y',
    num_jobs=-1,
)

Processing dataframe with multiple processes.


2024-09-12 06:25:15.749800: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-09-12 06:25:15.750217: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64


Finished preprocessing dataframe.
Finished forecasting.


In [13]:
timesfm_df

,unique_id,ds,timesfm,timesfm-q-0.1,timesfm-q-0.2,timesfm-q-0.3,timesfm-q-0.4,timesfm-q-0.5,timesfm-q-0.6,timesfm-q-0.7,timesfm-q-0.8,timesfm-q-0.9
0,Brno_1,2023-05-21,52436.609375,49662.824219,50383.757812,51344.703125,52027.324219,52436.609375,53140.347656,53792.757812,54678.453125,56299.164062
1,Brno_1,2023-05-28,52559.367188,49349.675781,50404.933594,51225.500000,51666.222656,52559.367188,53443.136719,53907.468750,55023.343750,56684.437500
2,Brno_1,2023-06-04,52322.390625,48711.375000,50041.492188,50975.210938,51623.976562,52322.390625,52879.125000,53731.390625,54832.246094,56584.828125
3,Brno_1,2023-06-11,52050.921875,48161.652344,49573.589844,50496.074219,51298.488281,52050.921875,52829.480469,53747.199219,54792.039062,56605.742188
4,Brno_1,2023-06-18,52101.187500,47960.386719,49465.656250,50615.945312,51349.457031,52101.187500,53200.058594,54237.136719,55458.328125,57699.246094
...,...,...,...,...,...,...,...,...,...,...,...,...
171,Prague_3,2024-02-18,32937.320312,28948.894531,30399.914062,31404.880859,32172.759766,32937.320312,33883.519531,34627.136719,35777.421875,37688.746094
172,Prague_3,2024-02-25,32928.566406,28879.326172,30338.128906,31305.267578,32081.765625,32928.566406,33710.824219,34779.343750,35670.214844,37564.847656
173,Prague_3,2024-03-03,32612.910156,28773.253906,30194.267578,31075.929688,31770.423828,32612.910156,33359.457031,34211.796875,35481.148438,37075.062500
174,Prague_3,2024-03-10,32369.390625,28600.896484,30077.099609,30839.482422,31851.195312,32369.390625,33653.054688,34409.015625,35719.351562,37384.417969


In [14]:
#save as csv
timesfm_df.to_csv("timesfm_weekly.csv")